In [1]:
from __future__ import division

from scipy import stats
from scipy.stats import pearsonr
from pandas.tools.plotting import scatter_matrix

from Bio import SeqIO
from Bio.Seq import Seq
import re

import numpy as np
import matplotlib.pyplot as plt
%pylab inline
import seaborn as sns
import pandas as pd
from pandas.io.parsers import *
import seaborn as sns
import os

Populating the interactive namespace from numpy and matplotlib


##Import data

In [2]:
samples = [f for f in os.listdir('Output/') if f.endswith('sort.sam')]
sample_root = ['%s_%s' %(sample.split('_')[6], sample.split('_')[8]) for sample in samples]
dict_sample_names = {'%s_%s' %(sample.split('_')[6], sample.split('_')[8]):sample for sample in samples}
dict_sample_names

{'DFSBLO002A_L001': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002A_S1_L001_R1_001_sort.sam',
 'DFSBLO002A_L002': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002A_S1_L002_R1_001_sort.sam',
 'DFSBLO002B_L001': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002B_S2_L001_R1_001_sort.sam',
 'DFSBLO002B_L002': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002B_S2_L002_R1_001_sort.sam',
 'DFSBLO002C_L001': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002C_S3_L001_R1_001_sort.sam',
 'DFSBLO002C_L002': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002C_S3_L002_R1_001_sort.sam',
 'DFSBLO002D_L001': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002D_S4_L001_R1_001_sort.sam',
 'DFSBLO002D_L002': 'mapped_dCas9_CP_trimmed_forBWA_14mer_DFSBLO002D_S4_L002_R1_001_sort.sam'}

In [3]:
sample_root

['DFSBLO002A_L001',
 'DFSBLO002A_L002',
 'DFSBLO002B_L001',
 'DFSBLO002B_L002',
 'DFSBLO002C_L001',
 'DFSBLO002C_L002',
 'DFSBLO002D_L001',
 'DFSBLO002D_L002']

In [4]:
dict_for_inFrame_gpby = {}

for sample in sample_root:
    dict_for_inFrame_gpby[sample] = pd.read_csv('Output/df_for_inFrame_gpby_%s.csv' %sample,
                                                names = ['POS','Counts'])

##Organize data

In [14]:
dict_for_inFrame_gpby_indexPos = {}

for key in dict_for_inFrame_gpby.iterkeys():
    dict_for_inFrame_gpby_indexPos[key] = dict_for_inFrame_gpby[key].set_index(keys = 'POS', drop = False)

In [16]:
dict_for_inFrame_gpby_indexPos['DFSBLO002A_L001']

,POS,Counts
POS,,
1,1,532
7,7,2764
13,13,32
22,22,2856
25,25,700
34,34,2300
49,49,3388
58,58,1
67,67,787


In [73]:
dict_init_pos = {i:0 for i in range(1,4167+1)}
df_pos_agg = pd.DataFrame(dict_init_pos, index = dict_init_pos.keys())
df_pos_agg['Index_Pos'] = df_pos_agg.index
df_pos_agg = df_pos_agg[['Index_Pos', 1]]
df_pos_agg.drop(labels = [1], axis = 1, inplace = True)

all_positions = []
for key in dict_for_inFrame_gpby_indexPos.iterkeys():
    all_positions += list(dict_for_inFrame_gpby_indexPos[key].index)

keep_indexes = list(set(all_positions))
keep_indexes.sort()

df_pos_agg_keep = df_pos_agg.ix[keep_indexes]

for df in [df_pos_agg, df_pos_agg_keep]:
    for col_name in dict_for_inFrame_gpby_indexPos.keys():
        df[col_name] = df['Index_Pos'].map(dict_for_inFrame_gpby_indexPos[col_name]['Counts'])
        df[col_name].fillna(value = 0, axis = 0, inplace = True)


In [75]:
df_pos_agg.describe()

,Index_Pos,DFSBLO002A_L002,DFSBLO002A_L001,DFSBLO002B_L002,DFSBLO002B_L001,DFSBLO002C_L001,DFSBLO002C_L002,DFSBLO002D_L001,DFSBLO002D_L002
count,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000,4167.000000
mean,2084.000000,553.017519,552.916727,3382.628510,3379.791937,2243.798176,2242.714663,3247.935685,3251.046796
std,1203.053615,3434.440650,3443.384208,67841.078634,67819.797255,49002.738308,48978.004918,69204.105997,69311.135258
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1042.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2084.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3125.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4167.000000,123508.000000,124480.000000,2892391.000000,2887662.000000,2735809.000000,2734924.000000,3697441.000000,3706686.000000


In [77]:
df_pos_agg_keep.describe()

,Index_Pos,DFSBLO002A_L002,DFSBLO002A_L001,DFSBLO002B_L002,DFSBLO002B_L001,DFSBLO002C_L001,DFSBLO002C_L002,DFSBLO002D_L001,DFSBLO002D_L002
count,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000
mean,2060.126126,3460.096096,3459.465465,21164.283784,21146.536036,14038.899399,14032.120120,20321.543544,20341.009009
std,1150.768645,7988.753577,8013.064879,168688.030826,168636.286731,121972.515672,121910.922194,172207.101176,172474.619717
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1130.500000,17.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2060.500000,1145.000000,1157.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2993.500000,3691.000000,3722.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,4144.000000,123508.000000,124480.000000,2892391.000000,2887662.000000,2735809.000000,2734924.000000,3697441.000000,3706686.000000


##Transform data for DESeq pipeline in R

In [122]:
df_pos_agg_keep.head()

,Index_Pos,DFSBLO002A_L002,DFSBLO002A_L001,DFSBLO002B_L002,DFSBLO002B_L001,DFSBLO002C_L001,DFSBLO002C_L002,DFSBLO002D_L001,DFSBLO002D_L002
1,1,560,532,22499,22693,2,5,29236,29212
7,7,2780,2764,292208,292248,233040,232452,532386,533110
10,10,0,0,4,2,2,0,2,1
13,13,29,32,3,2,4950,4747,13075,13026
19,19,1,0,0,0,0,0,0,0


In [123]:
df_ExportForDESeq = df_pos_agg_keep.copy()
df_ExportForDESeq['AminoAcidPos'] = df_ExportForDESeq['Index_Pos'].apply(lambda pos: (pos-1)/3+1)
df_ExportForDESeq.set_index(keys = 'AminoAcidPos', inplace = True, drop = True)
df_ExportForDESeq

,Index_Pos,DFSBLO002A_L002,DFSBLO002A_L001,DFSBLO002B_L002,DFSBLO002B_L001,DFSBLO002C_L001,DFSBLO002C_L002,DFSBLO002D_L001,DFSBLO002D_L002
AminoAcidPos,,,,,,,,,
1,1,560,532,22499,22693,2,5,29236,29212
3,7,2780,2764,292208,292248,233040,232452,532386,533110
4,10,0,0,4,2,2,0,2,1
5,13,29,32,3,2,4950,4747,13075,13026
7,19,1,0,0,0,0,0,0,0
8,22,2822,2856,0,0,0,1,0,0
9,25,702,700,0,0,0,0,0,0
11,31,0,0,1,0,0,0,0,0
12,34,2311,2300,0,0,0,1,0,0


In [124]:
new_cols = list(set([col.split('_')[0] for col in df_ExportForDESeq.columns if 'L' in col]))
new_cols.sort()

for col in new_cols:
    col1 = '%s_L001' %col
    col2 = '%s_L002' %col
    df_ExportForDESeq[col] = df_ExportForDESeq[col1] + df_ExportForDESeq[col2]

df_ExportForDESeq = df_ExportForDESeq[[col for col in df_ExportForDESeq.columns if not('_' in col)]]
df_ExportForDESeq

,DFSBLO002A,DFSBLO002B,DFSBLO002C,DFSBLO002D
AminoAcidPos,,,,
1,1092,45192,7,58448
3,5544,584456,465492,1065496
4,0,6,2,3
5,61,5,9697,26101
7,1,0,0,0
8,5678,0,1,0
9,1402,0,0,0
11,0,1,0,0
12,4611,0,1,0


In [125]:
df_ExportForDESeq.to_csv('df_ExportForDESeq.csv')